In [2]:
import os
import json
import random
import glob
from pathlib import Path
from ast import literal_eval

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as pe
from PIL import Image

import numpy as np
import pandas as pd

In [18]:
with open('./images/baguette/0.json', 'r') as f:
    labelme_load = json.load(f)

In [103]:
print(labelme_load.keys())
print(type(labelme_load['version']), type(labelme_load['flags']), type(labelme_load['shapes']), type(labelme_load['imagePath']), type(labelme_load['imageData']), type(labelme_load['imageHeight']), type(labelme_load['imageWidth']))
print(labelme_load['shapes'][0].keys())

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])
<class 'str'> <class 'dict'> <class 'list'> <class 'str'> <class 'str'> <class 'int'> <class 'int'>
dict_keys(['label', 'points', 'group_id', 'description', 'shape_type', 'flags', 'mask'])


In [118]:
data_dir = ".\\images"
data = glob.glob(os.path.join(data_dir + '/*/*.json'))
print(data[0], type(data[0]), len(data))

.\images\baguette\0.json <class 'str'> 600


In [119]:
make_coco = {'info': {}, 'licenses': [], 'images': [], 'annotations': [], 'categories': []}

for img_id in range(len(data)):

    with open(data[img_id], 'r') as f:
        load_json = json.load(f)

    make_coco['images'].append({'license': 0, 'file_name': data[img_id], 'coco_url': "", 'height': load_json['imageHeight'], 'width': load_json['imageWidth'], 'data_captured': 0, 'flickr_url': "", "id": img_id})

    for box_id, box in enumerate(load_json['shapes']):
        category_id_list = [category['name'] for category in make_coco['categories']]
        if box['label'] not in category_id_list:
            make_coco['categories'].append({'supercategory': 0, 'id': len(category_id_list), 'name': box['label']})
            category_id_list = [category['name'] for category in make_coco['categories']]

        x1, y1 = min(box['points'][0][0], box['points'][1][0]), min(box['points'][0][1], box['points'][1][1])
        x2, y2 = max(box['points'][0][0], box['points'][1][0]), max(box['points'][0][1], box['points'][1][1])
        w, h = x2 - x1, y2 - y1
        make_coco['annotations'].append({'segmentation': [], 'area': w * h, 'iscrowd': 0, 'image_id': img_id, 'bbox': [x1, y1, w, h], 'category_id': category_id_list.index(box['label']), 'id': box_id})

In [120]:
print(category_id_list)
print('----------images----------')
for i in range(3):
    idx = random.randint(0, len(data))
    print(make_coco['images'][idx])
print("----------annotations----------")
for i in range(10):
    idx = random.randint(900, 1200)
    print(make_coco['annotations'][idx])
print("----------categories----------")
print(make_coco['categories'])

['baguette', 'croissant', 'toast', 'iceamericano', 'powerade']
----------images----------
{'license': 0, 'file_name': '.\\images\\powerade\\60.json', 'coco_url': '', 'height': 168, 'width': 299, 'data_captured': 0, 'flickr_url': '', 'id': 437}
{'license': 0, 'file_name': '.\\images\\baguette\\61.json', 'coco_url': '', 'height': 225, 'width': 225, 'data_captured': 0, 'flickr_url': '', 'id': 78}
{'license': 0, 'file_name': '.\\images\\baguette\\98.json', 'coco_url': '', 'height': 194, 'width': 259, 'data_captured': 0, 'flickr_url': '', 'id': 118}
----------annotations----------
{'segmentation': [], 'area': 6402.245110386513, 'iscrowd': 0, 'image_id': 339, 'bbox': [100.7494004796163, 82.88369304556355, 68.10551558753, 94.00479616306956], 'category_id': 3, 'id': 0}
{'segmentation': [], 'area': 37037.856952899834, 'iscrowd': 0, 'image_id': 333, 'bbox': [16.805630026809652, 10.256032171581765, 164.0750670241287, 225.7372654155496], 'category_id': 3, 'id': 0}
{'segmentation': [], 'area': 7098

In [121]:
with open('make_coco.json', 'w') as make_file:
    json.dump(make_coco, make_file)

In [30]:
with open('labelme2coco.json', 'r') as f:
    load_json = json.load(f)

images_dir = ".\\images"
items = os.listdir(images_dir)
subdirectories = [item for item in items if os.path.isdir(os.path.join(images_dir, item))]
print(subdirectories)
for subdirectory in subdirectories:
    if subdirectory == 'croissant':
        break

    images = glob.glob(os.path.join(f"{images_dir}\\{subdirectory}\\*.jpg"))
    print(images[0])
    images_dic = {}

    indices = list(range(len(images)))
    random.shuffle(indices)
    split_point = int(0.2 * len(images))

    test_ids = indices[:split_point]
    print(test_ids)
    train_ids = indices[split_point:]

    for test_id in test_ids:
        for img in load_json['images']:
            if img['file_name'] == f'.\\images\\{subdirectory}\\{test_id}.jpg':
                img['license'] = 1

#images_id_list = [images['file_name'] for images in load_json['images']]
#print(images_id_list)

['baguette', 'croissant', 'iceamericano', 'powerade', 'toast']
.\images\baguette\0.jpg
[51, 18, 107, 106, 113, 99, 55, 53, 98, 118, 71, 115, 87, 11, 101, 41, 93, 88, 83, 72, 89, 26, 59, 68]


In [37]:
images = glob.glob(os.path.join(f".\\images\\*\\*.jpg"))
image_id = images[0]    # index에 맞는 image_id 가져오기
print(image_id)

.\images\baguette\0.jpg


In [52]:
items = os.listdir(images_dir)
subdirectories = [item for item in items if os.path.isdir(os.path.join(images_dir, item))]

for subdirectory in subdirectories:

    images = glob.glob(os.path.join(f"{images_dir}\\{subdirectory}\\*.jpg"))

    indices = list(range(len(images)))
    random.shuffle(indices)
    split_point = int(0.2 * len(images))

    test_ids = indices[:split_point]

    for test_id in test_ids:
        for img in load_json['images']:
            if img['file_name'] == f'.\\images\\{subdirectory}\\{test_id}.jpg':
                img['license'] = 1

train_json = {}
train_json['info'] = load_json['info']
train_json['licenses'] = load_json['licenses']
train_json['annotations'] = load_json['annotations']
train_json['categories'] = load_json['categories']
train_json['images'] = []
test_json = {}
test_json['info'] = load_json['info']
test_json['licenses'] = load_json['licenses']
test_json['annotations'] = load_json['annotations']
test_json['categories'] = load_json['categories']
test_json['images'] = []
for load_json_image in load_json['images']:
    if load_json_image['license'] == 1:
        test_json['images'].append(load_json_image)
    else:
        train_json['images'].append(load_json_image)

with open('train_json.json', 'w') as make_file:
    json.dump(train_json, make_file)
with open('test_json.json', 'w') as make_file:
    json.dump(test_json, make_file)

In [53]:
with open('labelme2coco.json', 'r') as f:
    load_json = json.load(f)
print(len(load_json['images']))

with open('train_json.json', 'r') as f:
    train_json = json.load(f)
print(len(train_json['images']))

with open('test_json.json', 'r') as f:
    test_json = json.load(f)
print(len(test_json['images']))

600
480
120


In [ ]:
class MeanAveragePrecision:
    def __init__(self, csv_path: os.PathLike) -> None:
        self.id_csv2coco = {}
        json_path = self.to_coco(csv_path)
        self.coco_gt = COCO(json_path)

        self.detections = []

    def to_coco(self, csv_path: os.PathLike) -> os.PathLike:
        df = pd.read_csv(csv_path)
        
        grouped = df.groupby(by='image_id')
        grouped_dict = {image_id: group for image_id, group in grouped}
        
        res = defaultdict(list)

        n_id = 0
        for image_id, (file_name, group) in enumerate(grouped_dict.items()):
            res['images'].append({
                'id': image_id,
                'width': 1024,
                'height': 1024,
                'file_name': f'{file_name}.jpg',
            })

            self.id_csv2coco[file_name] = image_id

            for _, row in group.iterrows():
                x1, y1, w, h = literal_eval(row['bbox'])
                res['annotations'].append({
                    'id': n_id,
                    'image_id': image_id,
                    'category_id': 1,
                    'area': w * h,
                    'bbox': [x1, y1, w, h],
                    'iscrowd': 0,
                })
                n_id += 1

        res['categories'].extend([{'id': 1, 'name': 'wheat'}])
            
        root_dir = os.path.split(csv_path)[0]
        save_path = os.path.join(root_dir, 'coco_annotations.json')
        with open(save_path, 'w') as f:
            json.dump(res, f)

        return save_path
    
    def update(self, preds, image_ids):
        for p, image_id in zip(preds, image_ids):
            p['boxes'][:, 2] = p['boxes'][:, 2] - p['boxes'][:, 0]
            p['boxes'][:, 3] = p['boxes'][:, 3] - p['boxes'][:, 1]
            p['boxes'] = p['boxes'].cpu().numpy()

            p['scores'] = p['scores'].cpu().numpy()
            p['labels'] = p['labels'].cpu().numpy()

            image_id = self.id_csv2coco[image_id]
            for b, l, s in zip(*p.values()):
                self.detections.append({
                    'image_id': image_id,
                    'category_id': l,
                    'bbox': b.tolist(),
                    'score': s
                })

    def reset(self):
        self.detections = []

    def compute(self):
        coco_dt = self.coco_gt.loadRes(self.detections)

        coco_eval = COCOeval(self.coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()